# Regression Model

### Importing Libraries

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import os
os.chdir("drive/MyDrive/DL_Project")

In [3]:
from Dataset import MyDataset
import torch
import torch.nn as nn
import torchvision.transforms as transforms

In [4]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Implementing Model

In [5]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Calculate the dimensions after convolution and pooling
        # Adjust the linear layer input size accordingly
        self.fc1 = nn.Linear(8 * 1024 * 1024, 64)  # 2048 / 2 / 2 = 1024
        self.fc2 = nn.Linear(64, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))

        # Flatten the tensor before passing it to fully connected layers
        x = x.view(-1, 8 * 1024 * 1024)

        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

model = CNN()
model.to(device)

CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8388608, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
)

### Setting Hyperparameters

In [6]:
num_epochs=10
batch_size=2
learning_rate=0.005

### Defining Loss Function and Optimizer

In [7]:
criterion=nn.MSELoss()
optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate)

### Training & Validating Model

In [8]:
transform = transforms.Compose([
    transforms.Resize((2048, 2048)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [9]:
train_dataset=MyDataset("Data/train_set.csv",transform=transform)
train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

In [10]:
valid_dataset=MyDataset("Data/validation_set.csv",transform=transform)
valid_dataloader=torch.utils.data.DataLoader(valid_dataset,batch_size=batch_size,shuffle=True)

In [11]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for i, (images, target) in enumerate(train_dataloader):
        images = images.reshape(-1,3,2048,2048).to(device)
        target = target.to(device).view(-1,1)

        optimizer.zero_grad()

        output = model(images)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        avg_loss=total_loss/len(train_dataloader)

    # Performing validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0

        for i, (val_images, val_target) in enumerate(valid_dataloader):
            val_images = val_images.reshape(-1,3,2048,2048).to(device)
            val_target = val_target.to(device).view(-1,1)

            val_output = model(val_images)
            val_loss += criterion(val_output, val_target).item()

            average_val_loss = val_loss / len(valid_dataloader)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_loss:.4f}, Validation Loss: {average_val_loss:.4f}')

Epoch [1/10], Training Loss: 1960022.7293, Validation Loss: 102.4113
Epoch [2/10], Training Loss: 1625.7378, Validation Loss: 1.8912
Epoch [3/10], Training Loss: 23.8897, Validation Loss: 1.3560
Epoch [4/10], Training Loss: 6.7857, Validation Loss: 1.2768
Epoch [5/10], Training Loss: 14.1698, Validation Loss: 1.4368


KeyboardInterrupt: ignored

### Saving Model Weights

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
model.load_state_dict(torch.load('model_weights.pth'))

### Testing Model

In [12]:
test_dataset=MyDataset("Data/test_set.csv",transform=transform)
test_dataloader=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [14]:
predicted=[]
actual=[]
model.eval()
with torch.no_grad():
    test_loss = 0.0

    for i, (test_images, test_target) in enumerate(test_dataloader):
        test_images = test_images.reshape(-1,3,2048,2048).to(device)
        test_target = test_target.to(device).view(-1,1)

        test_output = model(test_images)
        test_loss += criterion(test_output, test_target).item()

        average_test_loss = test_loss / len(test_dataloader)

        predicted.append(test_output)
        actual.append(test_target)

    print(f'Average Loss: {average_test_loss}')

Average Loss: 1.769939105923253


In [15]:
for x,y in zip(predicted,actual):
  for i,j in zip(x,y):
    print(i.item(),"  ",j.item())

1.5263001918792725    3.4549999237060547
1.0516618490219116    2.125
1.2205700874328613    1.8350000381469727
1.2819716930389404    1.940000057220459
1.2769827842712402    2.2300000190734863
1.515235185623169    3.0999999046325684
1.2814452648162842    1.725000023841858
1.2978097200393677    2.115000009536743
1.3272441625595093    2.3399999141693115
1.45932936668396    1.0700000524520874
1.1825096607208252    2.0299999713897705
1.2713806629180908    1.350000023841858
0.7964275479316711    2.2950000762939453
1.200515627861023    2.115000009536743
1.2247408628463745    2.7100000381469727
1.289649486541748    2.5350000858306885
1.049232840538025    2.7100000381469727
1.3806689977645874    3.180000066757202
1.492409586906433    4.050000190734863
1.390131950378418    3.549999952316284
1.240668773651123    0.7649999856948853
1.2609940767288208    3.940000057220459
1.1518058776855469    0.28999999165534973
1.263269066810608    0.6000000238418579
1.4322949647903442    0.17000000178813934
1.068